## Precision and accuracy of eyetracking VSHD (Arrington ViewPoint)
### This notebook reads in a folder of datafiles made using the "Precision and Accuracy.sln" program, and prints a table with the measurements for each one
#### Imports libraries:

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt

#### Folder directory:

In [2]:
directory = 'data'

#### Function that calculates standard deviation of a list of 2D angles 

In [3]:
def calculateAngleStd(azimuthList, elevationList):
    azimuthList = azimuthList.reset_index(drop=True)
    elevationList = elevationList.reset_index(drop=True)
    
    
    meanAzimuth , meanElevation = azimuthList.mean(), elevationList.mean()
    meanAngle=(meanAzimuth, meanElevation)
    distanceList = []
    for i in range(len(azimuthList)):
        
        d = calculateDistance((azimuthList[i], elevationList[i]), meanAngle)

        distanceList.append(d)
    
    
    a=0
    for d in distanceList:
        a+=d**2
    std = math.sqrt(a/len(distanceList))

    
    
    return std

#### Function that calculates angular distance between two angles

In [4]:
def calculateDistance(a, b):
    AB = (a[0]-b[0], a[1] - b[1])

    d_AB = math.sqrt(AB[0]**2 + AB[1]**2)

    return d_AB

    

#### Function that calculates the precision of points in a DataFrame by measuring the distance between each point and their respective succesive points, then taking the root mean square (RMS) of these distances

In [5]:
def calculatePrecision(df): 
    gazePoints = []
    for i in range(len(df)):

        gazePoints.append((df["Azimuth"].reset_index(drop=True)[i], df["Elevation"].reset_index(drop=True)[i]))


    distanceList = []
    for i in range(len(gazePoints)-1):
        d = calculateDistance(gazePoints[i], gazePoints[i+1])

        distanceList.append(d)

    a=0
    for d in distanceList:
        a+=d**2
    precision = math.sqrt(a/len(distanceList))
    
    return precision


#### This function extracts gaze data from all of the data measured for each of the points shown on the screens
It finds the 18 succesive datapoints (300ms) in the data with the lowest standard deviation, and tries to extend the data one sample at a time in each direction until the standard deviation gets worse.

In [6]:
def getGazeData(df):  
    dropStart = 18

    angleData = df[["Azimuth","Elevation"]][dropStart:].reset_index(drop=True)

    gazeData = angleData[:18]

    for i in range(len(df)-17-dropStart):
        newGazeData = angleData[i:i+18] 
        if calculateAngleStd(gazeData["Azimuth"], gazeData["Elevation"]) > calculateAngleStd(newGazeData["Azimuth"], newGazeData["Elevation"]):
            gazeData = newGazeData

    
    
    
    #Tries to extend
        
    x=0
    while True:
        
        indexToAdd = gazeData.index[-1] + 1
        
        
        if indexToAdd in angleData.index:
            extGazeData = gazeData.append(angleData.iloc[indexToAdd]).sort_index()
        else:
            break
        
        
        
        if calculateAngleStd(gazeData["Azimuth"], gazeData["Elevation"]) > calculateAngleStd(extGazeData["Azimuth"], extGazeData["Elevation"]):
            gazeData = extGazeData
        else:
            break
        x+=1
        if x > 160:
            break
    
    while True:
        
        indexToAdd = gazeData.index[0] -1
        
       
        if indexToAdd in angleData.index:
            extGazeData = gazeData.append(angleData.iloc[indexToAdd]).sort_index()
        else:
            break

        
        if calculateAngleStd(gazeData["Azimuth"], gazeData["Elevation"]) > calculateAngleStd(extGazeData["Azimuth"], extGazeData["Elevation"]):
            gazeData = extGazeData
        else:
            break
        x+=1
        if x > 160:
            break
    return gazeData
    

#### Function that converts screen coordinates into gaze angle
Middle of screen is gaze angle = 0 and x=0.5 y=0.5

In [7]:
def calculateAngle(screenX, screenY, fovHorizontal=52.1, fovVertical=34):
    x = (screenX*2) - 1
    y = (screenY*2) - 1

    azimuth = math.atan(x*math.tan(fovHorizontal*math.pi/360))
    elevation = math.atan(y*math.tan(fovVertical*math.pi/360))

    
    return azimuth*180/math.pi, elevation*180/math.pi

#This function assumes a linear relationship between screen coordinates and gaze angle, and should therefore be less accurate than the above function
def calculateAngle2(screenX, screenY, fovHorizontal=52.1, fovVertical=34):
    x = (screenX*2) - 1
    y = (screenY*2) - 1

    azimuth = x*fovHorizontal/2
    elevation = y*fovVertical/2

    
    return azimuth, elevation


print(calculateAngle(0.84,0.81))



(18.38644820009973, 10.733247445709274)


#### Sets list of the true point values for comparison later

In [8]:
truePointValues = [(0.30, 0.75),(0.70, 0.25),(0.30, 0.25),(0.70, 0.75),(0.40, 0.60),(0.60, 0.60),(0.40, 0.40),(0.60, 0.40),(0.50, 0.425),(0.50, 0.75),(0.50, 0.25),(0.30, 0.50),(0.50, 0.35),(0.70, 0.50),(0.50, 0.65),(0.50, 0.50)]
truePointAngles = []

for point in truePointValues:
    angle = calculateAngle(point[0], point[1])
    truePointAngles.append(angle)
print(truePointAngles)

[(-11.06320230741821, 8.691257073883165), (11.063202307418205, -8.691257073883165), (-11.06320230741821, -8.691257073883165), (11.063202307418205, 8.691257073883165), (-5.583644357117053, 3.4990590634325542), (5.583644357117053, 3.4990590634325542), (-5.583644357117053, -3.4990590634325542), (5.583644357117053, -3.4990590634325542), (0.0, -2.6257219671274514), (0.0, 8.691257073883165), (0.0, -8.691257073883165), (-11.06320230741821, 0.0), (0.0, -5.240461161882563), (11.063202307418205, 0.0), (0.0, 5.240461161882563), (0.0, 0.0)]


#### Algorithm that finds the precision and accuracy-data, then plots it


In [9]:
finalAccuracyList=[]
finalPrecisionList=[]




# iterate over files in the directory
for filename in os.scandir(directory):
    with open(filename) as rawData: 
        lineNumber = 0
        stimulusPointNr = 0
        unwantedLabels = ["6","ADT", "ACX", "ACY","ARI", "APW", "APH","AQU","AFX","BDT","BCX","BCY","BRI","BPW","BPH","BQU","BFX","GPX","GPY","GPZ","CNT"]
        truePointValues = [(0.30, 0.75),(0.70, 0.25),(0.30, 0.25),(0.70, 0.75),(0.40, 0.60),(0.60, 0.60),(0.40, 0.40),(0.60, 0.40),(0.50, 0.425),(0.50, 0.75),(0.50, 0.25),(0.30, 0.50),(0.50, 0.35),(0.70, 0.50),(0.50, 0.65),(0.50, 0.50)]
        currentPointData = 0
        del currentPointData
        dropFirstRows = list(range(0,60))


        for line in rawData:
            splittedLine =line.split()


            #If index of raw data is 6 it adds as columns
            if splittedLine[0] == "6":
                splittedLine.pop(27) #pops mrk label
                DfColumns = splittedLine
                formattedData = pd.DataFrame(columns=splittedLine)

                #Defines dataframes for measured data for all the stimulus points
                point1 = pd.DataFrame(columns=DfColumns)
                point2 = pd.DataFrame(columns=DfColumns)
                point3 = pd.DataFrame(columns=DfColumns)
                point4 = pd.DataFrame(columns=DfColumns)
                point5 = pd.DataFrame(columns=DfColumns)
                point6 = pd.DataFrame(columns=DfColumns)
                point7 = pd.DataFrame(columns=DfColumns)
                point8 = pd.DataFrame(columns=DfColumns)
                point9 = pd.DataFrame(columns=DfColumns)
                point10 = pd.DataFrame(columns=DfColumns)
                point11= pd.DataFrame(columns=DfColumns)
                point12 = pd.DataFrame(columns=DfColumns)
                point13 = pd.DataFrame(columns=DfColumns)
                point14 = pd.DataFrame(columns=DfColumns)
                point15 = pd.DataFrame(columns=DfColumns)
                point16 = pd.DataFrame(columns=DfColumns)

                stimulusPointData = [point1, point2, point3, point4, point5, point6, point7, point8, point9, point10, point11, point12, point13, point14, point15, point16]

            elif splittedLine[0] == "16":
            
                if splittedLine[1][0]=="0":
                    print(splittedLine)
                #if stimulusPointNr >= 0: #Does so that it doesnt start appending the first lines to the last dataframe
                else:
                    currentPointData = stimulusPointData[stimulusPointNr]
                    stimulusPointNr +=1
           


            #All rows of data have index 10 in the raw data, some have marks though and are therefore 1 column longer,
            #this checks if length is 28 or 27. If its 28, pop last element and append to DF otherwise just append to DF.
            elif splittedLine[0] == "10":
                try: 
                    splittedLine.pop(27)
                    currentPointData.loc[len(currentPointData.index)] = splittedLine
                except IndexError:
                    try:
                        currentPointData.loc[len(currentPointData.index)] = splittedLine
                    except NameError:
                        pass
                except NameError:
                    pass

        #Adds avg gaze position of both eyes and removes unneccesary columns
    i=0

    for df in stimulusPointData:

        for label in unwantedLabels:
            df.pop(label)
            df=df.astype(float)
        df["AverageABX"]=df[["ALX","BLX"]].mean(axis=1, numeric_only=True)
        df["AverageABY"]=df[["ALY","BLY"]].mean(axis=1, numeric_only=True)


        azimuthList = []
        elevationList = []

        for index in range(len(df["AverageABX"])):
            azimuth, elevation = calculateAngle(df["AverageABX"][index], df["AverageABY"][index])

            azimuthList.append(azimuth)
            elevationList.append(elevation)

        df["Azimuth"] = azimuthList
        df["Elevation"] = elevationList

        stimulusPointData[i] = df
        i+=1


    gazeDataList = list(stimulusPointData)

    i=0
    for df in stimulusPointData:
        gazeDataList[i]=getGazeData(df)
        i+=1


    sumPrc=0
    for df in gazeDataList:
        sumPrc+=calculatePrecision(df)
    avgPrc=sumPrc/len(gazeDataList)
    finalPrecisionList.append(avgPrc)

    accuracyList = []

    for stimPointIndex in range(len(truePointAngles)):

        df=gazeDataList[stimPointIndex]

        df=df.reset_index(drop=True)
        distanceList = []
        gazePoints=[]
        for i in range(len(df)):
            gazePoints.append((df["Azimuth"].reset_index(drop=True)[i], df["Elevation"].reset_index(drop=True)[i]))

            d = calculateDistance(gazePoints[i],truePointAngles[stimPointIndex])
            
            distanceList.append(d)
            

        accuracy = pd.Series(distanceList).mean()

        if f"(del_{len(accuracyList)}" in str(filename):
            print(f"Point {len(accuracyList)} deleted from {str(filename)}")
        else:
            accuracyList.append(accuracy)
        
            


    finalAccuracyList.append(pd.Series(accuracyList).mean())

    #If set to true all measured points will be plotted next to true points
    if False:
        print(str(filename))    
        for frame in range(16):
            fig=plt
            fig.clf()
            fig.plot(gazeDataList[frame]["Azimuth"], gazeDataList[frame]["Elevation"])
            fig.plot(truePointAngles[frame][0], truePointAngles[frame][1], marker=".", color="red")
            fig.xlim(-26.05, 26.05)
            fig.ylim(-17, 17)
            print(frame)
            fig.show()

print(finalPrecisionList, "Average precision:",pd.Series(finalPrecisionList).mean())
print(finalAccuracyList, "Average accuracy:",pd.Series(finalAccuracyList).mean())        

['16', '0.000000', 'C:\\Users\\Administrator\\Documents\\PrecisionAndAccuracy\\StimulusPoints_bmp\\16_96.bmp']
[0.08481547833763654] Average precision: 0.08481547833763654
[0.7891532981261509] Average accuracy: 0.7891532981261509


#### Puts data in Data Frame

In [10]:
fileNameList=[]
for file in os.scandir(directory):
    fileNameList.append(str(file))
pd.DataFrame(data=np.array([fileNameList, finalAccuracyList, finalPrecisionList]).T, columns=("File","Accuracy (degrees Mean Error)", "Precision (degrees RMS)"))

,File,Accuracy (degrees Mean Error),Precision (degrees RMS)
0,<DirEntry 'myfile.txt'>,0.7891532981261509,0.08481547833763654


In [11]:
print("Final mean precision:",np.array(finalPrecisionList).mean())

Final mean precision: 0.08481547833763654


In [13]:
print("Final mean accuracy:",np.array(finalAccuracyList).mean())

Final mean accuracy: 0.7891532981261509
